In [9]:
import pandas as pd
import numpy as np

In [10]:
train = pd.read_csv('train_df_noun.csv',)
test = pd.read_csv('test_df_noun.csv')

In [11]:
drop_indices_1 = [40456]
all_drop_indices = drop_indices_1
train = train.drop(all_drop_indices)

In [12]:
train['제목_키워드'] = train['제목_키워드'].replace('용인시','용인',regex=True)
test['제목_키워드'] = test['제목_키워드'].replace('용인시','용인',regex=True)

In [13]:
train['제목_키워드'] = train['제목_키워드'].replace('배드민턴','스포츠',regex=True)
train['제목_키워드'] = train['제목_키워드'].replace('야구','스포츠',regex=True)
train['제목_키워드'] = train['제목_키워드'].replace('배구','스포츠',regex=True)
train['제목_키워드'] = train['제목_키워드'].replace('골프','스포츠',regex=True)
train['제목_키워드'] = train['제목_키워드'].replace('럭비단','스포츠',regex=True)

test['제목_키워드'] = test['제목_키워드'].replace('배드민턴','스포츠',regex=True)
test['제목_키워드'] = test['제목_키워드'].replace('야구','스포츠',regex=True)
test['제목_키워드'] = test['제목_키워드'].replace('배구','스포츠',regex=True)
test['제목_키워드'] = test['제목_키워드'].replace('골프','스포츠',regex=True)
test['제목_키워드'] = test['제목_키워드'].replace('럭비단','스포츠',regex=True)


In [14]:
from collections import Counter

# 모든 키워드를 공백으로 분리하고 하나의 리스트로 만들기
all_keywords = ' '.join(train['제목_키워드']).split()

# 각 단어의 빈도 수 계산
keyword_counts = Counter(all_keywords)

# 가장 많이 나오는 단어 5개 추출
most_common_keywords = keyword_counts.most_common(5)

# 결과 출력
print(most_common_keywords)

[('용인', 125740), ('사업', 79764), ('지역', 66696), ('지원', 61708), ('반도체', 57839)]


In [15]:
train['제목_키워드'] = train['제목_키워드'].apply(lambda x: ' '.join([word for word in x.split(' ') if len(word) > 1]))
test['제목_키워드'] = test['제목_키워드'].apply(lambda x: ' '.join([word for word in x.split(' ') if len(word) > 1]))

In [16]:
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
# 키워드와 본문 데이터를 각각 벡터화
vectorizer_keyword = CountVectorizer(analyzer = 'word',max_features = 5000)

X_keyword = vectorizer_keyword.fit_transform(train['제목_키워드'])  # 제목_키워드 벡터화

# 두 벡터를 결합
X = X_keyword
y = train['분류']


In [17]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
# 랜덤 시드 및 데이터 분할
RANDOM_SEED = 42
TEST_SPLIT = 0.25

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)


In [18]:
import joblib
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import f1_score, classification_report

rf = joblib.load('rf_model.pkl')
sgd = joblib.load('sgd_model.pkl')
lr = joblib.load('logistic_model.pkl')

In [19]:
# 앙상블 모델 생성 (Voting Classifier)
ensemble_model = VotingClassifier(estimators=[('log_reg', lr), ('sgd', sgd), ('rf', rf)], voting='soft')

# 앙상블 모델 학습
ensemble_model.fit(X_train, y_train)

# 예측 수행
y_pred = ensemble_model.predict(X_test)

# 성능 평가 - Macro F1 Score 계산
macro_f1 = f1_score(y_test, y_pred, average='macro')
report = classification_report(y_test,y_pred)
# 최적 시드와 해당 성능 출력
print(f"Macro F1 Score: {macro_f1}")
print("Classification Report:\n", report)


C:\Users\yeonn\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Macro F1 Score: 0.7030646645790115
Classification Report:
                precision    recall  f1-score   support

IT_과학:IT_과학일반       0.40      0.36      0.38        61
     IT_과학:과학       0.85      0.63      0.72        54
    IT_과학:모바일       0.80      0.77      0.78       133
     IT_과학:보안       0.86      0.52      0.65        23
IT_과학:인터넷_SNS       0.63      0.53      0.58        60
    IT_과학:콘텐츠       0.74      0.62      0.68        40
      경제:경제일반       0.53      0.45      0.49       155
    경제:금융_재테크       0.89      0.71      0.79        82
        경제:무역       0.87      0.83      0.85        93
       경제:반도체       0.90      0.87      0.89       577
       경제:부동산       0.85      0.90      0.87       862
     경제:산업_기업       0.62      0.65      0.64       177
    경제:서비스_쇼핑       0.73      0.68      0.71        97
        경제:외환       1.00      1.00      1.00         2
        경제:유통       0.76      0.73      0.75       146
       경제:자동차       0.78      0.76      0.77       157
     

In [20]:
ensemble_model.fit(X, y)

C:\Users\yeonn\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('log_reg',
                              LogisticRegression(C=1.623776739188721,
                                                 random_state=65)),
                             ('sgd',
                              SGDClassifier(loss='log_loss', max_iter=2000,
                                            random_state=65, tol=1e-05)),
                             ('rf', RandomForestClassifier(random_state=65))],
                 voting='soft')

In [21]:
test_keyword = vectorizer_keyword.transform(test['제목_키워드'])  # 키워드 벡터화

In [22]:
test_pred = ensemble_model.predict(test_keyword)

In [23]:
submission = pd.read_csv('sample_submission_hakaton.csv')
submission['분류'] = test_pred
submission.to_csv('hakaton_submission_final.csv',encoding='UTF-8-sig', index=False)

In [24]:
submission

,ID,분류
0,TEST_00000,사회:사회일반
1,TEST_00001,사회:사회일반
2,TEST_00002,정치:행정_자치
3,TEST_00003,경제:취업_창업
4,TEST_00004,지역
...,...,...
23400,TEST_23400,사회:사회일반
23401,TEST_23401,지역
23402,TEST_23402,지역
23403,TEST_23403,지역
